In [ ]:
import visdom
from keras.callbacks import Callback
import numpy as np

class VisdomCallback(Callback):
    def __init__(self, env='main', valid_gen=None, num_images=3):
        self.viz = visdom.Visdom()
        self.env = env
        self.valid_gen = valid_gen
        self.num_images = num_images
        self.epochs = []
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []
        
        self.win_loss = None
        self.win_acc = None
        self.win_images = None

    def on_epoch_end(self, epoch, logs=None):
        self.epochs.append(epoch)
        self.train_losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.train_accuracies.append(logs.get('accuracy'))
        self.val_accuracies.append(logs.get('val_accuracy'))

        self.update_visdom()
        self.visualize_predictions()

    def update_visdom(self):
        if self.win_loss is None:
            self.win_loss = self.viz.line(
                X=self.epochs, Y=self.train_losses, env=self.env,
                opts=dict(title="Loss", xlabel="Epochs", ylabel="Loss", legend=["Train Loss", "Val Loss"]))
            self.viz.line(
                X=self.epochs, Y=self.val_losses, env=self.env, win=self.win_loss, update='append')
        else:
            self.viz.line(
                X=[self.epochs[-1]], Y=[self.train_losses[-1]], env=self.env, win=self.win_loss, update='append')
            self.viz.line(
                X=[self.epochs[-1]], Y=[self.val_losses[-1]], env=self.env, win=self.win_loss, update='append')

        if self.win_acc is None:
            self.win_acc = self.viz.line(
                X=self.epochs, Y=self.train_accuracies, env=self.env,
                opts=dict(title="Accuracy", xlabel="Epochs", ylabel="Accuracy", legend=["Train Accuracy", "Val Accuracy"]))
            self.viz.line(
                X=self.epochs, Y=self.val_accuracies, env=self.env, win=self.win_acc, update='append')
        else:
            self.viz.line(
                X=[self.epochs[-1]], Y=[self.train_accuracies[-1]], env=self.env, win=self.win_acc, update='append')
            self.viz.line(
                X=[self.epochs[-1]], Y=[self.val_accuracies[-1]], env=self.env, win=self.win_acc, update='append')

    def visualize_predictions(self):
        if self.valid_gen is None:
            return

        x, y_true = next(self.valid_gen)
        y_pred = self.model.predict(x)

        for i in range(self.num_images):
            img = np.uint8(x[i] * 255)  # Scale to [0, 255] and convert to uint8
            img = np.squeeze(img)  # Remove single-dimensional entries from the shape
            img = np.expand_dims(img, axis=0)  # Add channel dimension

            true_label = np.argmax(y_true[i])
            pred_label = np.argmax(y_pred[i])

            self.viz.image(img, win='image_{}'.format(i),
                           opts=dict(title="Image {} - True: {} - Pred: {}".format(i, true_label, pred_label)))

# Inisialisasi callback Visdom
visdom_callback = VisdomCallback(env='training_monitor', valid_gen=valid_gen) #Perhatikan Data Validasi